In [ ]:
import csv
import pandas as pd
from datetime import datetime
from pydantic import BaseModel
from enum import Enum, auto
from typing import List
import random
import names


In [ ]:
# Generalized Reduced Gradient Algorithm 

import numpy as np
import matplotlib.pyplot as plt
from sympy import *

def generalized_reduced_gradient():

	x1, x2, x3 = symbols('x1 x2 x3')
	xvars = [x1, x2, x3]

	fx = 4 * x1 - x2 ** 2 + x3 ** 2 - 12				# Function to be minimized
	hxs = [20 - x1 ** 2 - x2 ** 2, x1 + x3 - 7]			# Constraints to be obeyed
	alpha_0 = 1											# Parameter initializations
	gamma = 0.4
	max_iter = 100
	max_outer_iter = 50
	eps_1, eps_2, eps_3 = 0.001, 0.001, 0.001

	xcurr = np.array([2, 4, 5])							# Starting solution

	dfx = np.array([diff(fx, xvar) for xvar in xvars])
	dhxs = np.array([[diff(hx, xvar) for xvar in xvars] for hx in hxs])
	nonbasic_vars = len(xvars) - len(hxs)
	opt_sols = []

	for outer_iter in range(max_outer_iter):

		print(f"Outer loop iteration: {outer_iter + 1}, optimal solution: {xcurr}")
		opt_sols.append(fx.subs(zip(xvars, xcurr)))

		# Step 1

		delta_f = np.array([df.subs(zip(xvars, xcurr)) for df in dfx])
		delta_h = np.array([[dh.subs(zip(xvars, xcurr)) for dh in dhx] for dhx in dhxs])		# Value of h'_i(xcurr) for all i
		J = np.array([dhx[nonbasic_vars:] for dhx in delta_h])									# Computation of J and C matrices
		C = np.array([dhx[:nonbasic_vars] for dhx in delta_h])
		delta_f_bar = delta_f[nonbasic_vars:]
		delta_f_cap = delta_f[:nonbasic_vars]

		J_inv = np.linalg.inv(np.array(J, dtype=float))
		delta_f_tilde = delta_f_cap - delta_f_bar.dot(J_inv.dot(C))

		# Step 2

		if abs(delta_f_tilde[0]) <= eps_1:
			break

		d_bar = - delta_f_tilde.T 									# Direction of search in current iteration
		d_cap = - J_inv.dot(C.dot(d_bar))
		d = np.concatenate((d_bar, d_cap)).T

		# Step 3

		alpha = alpha_0

		while alpha > 0.001:

			print (f"Alpha value: {alpha}")

			# Step 3(a)

			v = xcurr.T + alpha * d
			v_bar = v[:nonbasic_vars]
			v_cap = v[nonbasic_vars:]
			flag = False

			for iter in range(max_iter):
				print (f"Iteration: {iter + 1}, optimal solution obtained at x = {v}")
				h = np.array([hx.subs(zip(xvars, v)) for hx in hxs])
				if all([abs(h_i) < eps_2 for h_i in h]):				# Check if candidate satisfies all constraints
					if fx.subs(zip(xvars, xcurr)) <= fx.subs(zip(xvars, v)):
						alpha = alpha * gamma
						break
					else:
						xcurr = v 						# Obtained a candidate better than the current optimal solution
						flag = True
						break

				# Step 3(b)

				delta_h_v = np.array([[dh.subs(zip(xvars, v)) for dh in dhx] for dhx in dhxs])
				J_inv_v = np.linalg.inv(np.array([dhx[nonbasic_vars:] for dhx in delta_h_v], dtype=float))
				v_next_cap = v_cap - J_inv_v.dot(h)

				# Step 3(c)

				if abs(np.linalg.norm(np.array(v_cap - v_next_cap, dtype=float), 1)) > eps_3:
					v_cap = v_next_cap
					v = np.concatenate((v_bar, v_cap))
				else:
					v_cap = v_next_cap
					v = np.concatenate((v_bar, v_cap))
					h = np.array([hx.subs(zip(xvars, v)) for hx in hxs])
					if all([abs(h_i) < eps_2 for h_i in h]):

						# Step 3(d)

						if fx.subs(zip(xvars, xcurr)) <= fx.subs(zip(xvars, v)):
							alpha = alpha * gamma				# Search for lower values of alpha
							break
						else:
							xcurr = v
							flag = True
							break
					else:
						alpha = alpha * gamma
						break
			
			if flag == True:
				break

	print (f"Final solution obtained is: {xcurr}")
	print (f"Value of the function at this point: {fx.subs(zip(xvars, xcurr))}")

	plt.plot(opt_sols, 'ro')								# Plot the solutions obtained after every iteration
	plt.show()

if __name__ == '__main__':
	generalized_reduced_gradient()

In [ ]:


class Division(Enum):
    U05B = auto()
    U06B = auto()
    U08B = auto()
    U10B = auto()
    U12B = auto()
    U14B = auto()
    U16B = auto()
    U19B = auto()
    U05G = auto()
    U06G = auto()
    U08G = auto()
    U10G = auto()
    U12G = auto()
    U14G = auto()
    U16G = auto()

class DivisionConfig(BaseModel):
    division: Division
    teams: int
    players_per_team: int
    
class Score(BaseModel):
    player_id: int
    player_name: str
    player_division: Division
    player_score: int = 0


def load_data_from_file() -> List[Score]:
    with open('data.csv', 'r') as file:
        reader = csv.reader(file)
        scores = List[Score]
        for each_row in reader:
           scores.append(Score(*each_row) )

g_player_id: int = 8000

def GetPlayerId():
    global g_player_id
    g_player_id = g_player_id + 1
    return g_player_id

def generate_random_data_for_division(config: DivisionConfig):
    
    # print(config)
    for t in range(0, config.teams):
        for x in range(1, config.players_per_team+1):
            name = names.get_full_name()
            score = random.randint(20,100)
            player_id = GetPlayerId()
            yield  (player_id,name,config.division.name,score )
            


In [ ]:
config = [  DivisionConfig(division=Division.U05B, teams=8, players_per_team=7)
            # DivisionConfig(division=Division.U05G, teams=4, players_per_team=5),
            # DivisionConfig(division=Division.U06B, teams=8, players_per_team=5),
            # DivisionConfig(division=Division.U06G, teams=4, players_per_team=5),
            # DivisionConfig(division=Division.U08B, teams=20, players_per_team=5),
            # DivisionConfig(division=Division.U08G, teams=16, players_per_team=5),
            # DivisionConfig(division=Division.U10B, teams=12, players_per_team=5),
            # DivisionConfig(division=Division.U10G, teams=10, players_per_team=5),
            # DivisionConfig(division=Division.U12B, teams=8, players_per_team=5),
            # DivisionConfig(division=Division.U12G, teams=8, players_per_team=5),
            # DivisionConfig(division=Division.U14B, teams=22, players_per_team=5),
            # DivisionConfig(division=Division.U14G, teams=18, players_per_team=5),
            # DivisionConfig(division=Division.U16B, teams=0, players_per_team=0),
            # DivisionConfig(division=Division.U16G, teams=18, players_per_team=5),
            # DivisionConfig(division=Division.U19B, teams=0, players_per_team=0)
            ]

In [ ]:
from typing  import Tuple


def prepare(d: DivisionConfig, df: pd.DataFrame) -> Tuple[pd.DataFrame, float]:
   
    df["score_reduced"] = df["player_score"] / 100
    df["mean"] = df["score_reduced"].mean()
    df["mean_minus_score"] =  df["score_reduced"] - df["mean"] 
    
    df.drop(["player_name", "player_score"], axis=1, inplace=True)
    
    for x in range(d.teams):
        df[f"Team {x}"] = 0
    
    score_mean = df["mean"].first
    
    return (df, score_mean)
    

In [114]:
import scipy as sp
import scipy.optimize as optimize
import numpy as np
import numpy.matlib as mb

In [ ]:
def solver_a(d:DivisionConfig, dfp:pd.DataFrame, mean: float):
    players = dfp.sort_values(by=["mean_minus_score"], axis=0, ascending=False, ignore_index=True, key=None)
    
    for index, row in players.iterrows():
        is_assigned = False
        for t in range(d.teams):
            if row[f"Team {t}"]:
                

In [ ]:
# constraints:
# 1. All players are assigned to one team 
# 2. Sum of players scores in the team <= mean of all player scores

In [ ]:
from audioop import avg
import os
from matplotlib import pyplot
# from numpy import where

test_data_path = os.path.abspath(os.getcwd())
 
for d in config:
        print(f"\t --> {d.division}")
        players = [x for x in generate_random_data_for_division(d) ]
        
        df = (pd.DataFrame(data=players, columns=["player_number", "player_name",  "player_division", "player_score"])
                .sort_values(by=["player_division", "player_score"], axis=0, ascending=False, ignore_index=True, key=None)
                )
        
        
        dfp, mean = prepare(d, df)
        solver_a(d, dfp, mean)
        
                
        # df.to_excel(test_data_path+f"\\test_data_{d.division}.xlsx")